归一化：目的是对节点的特征向量做行L1归一化，每一行的和是1，具体实现是创建了一个每一行和的倒数的对角矩阵乘以特征向量（和度的-1乘X获得邻居求和的平均值同理）

In [19]:
import scipy.sparse as sp
data = [1, 1, 2]
row = [0, 1, 1]
col = [0, 1, 2]
matrix = sp.coo_matrix((data, (row, col)), shape=(3, 3))
matrix.todense()

matrix([[1, 0, 0],
        [0, 1, 2],
        [0, 0, 0]])

In [20]:
import numpy as np
rowsum  = np.array(matrix.sum(1))
rowsum

array([[1],
       [3],
       [0]], dtype=int32)

np.power cannot be used in float，change it to np.float_power

In [33]:
r_inv = np.float_power(rowsum, -1).flatten()
r_inv[np.isinf(r_inv)] = 0.
r_inv

C:\Users\JackLu\AppData\Local\Temp\ipykernel_3272\3059307869.py:1: RuntimeWarning: divide by zero encountered in float_power
  r_inv = np.float_power(rowsum, -1).flatten()


array([1.        , 0.33333333, 0.        ])

In [34]:
r_mat_inv = sp.diags(r_inv)
r_mat_inv.toarray()

array([[1.        , 0.        , 0.        ],
       [0.        , 0.33333333, 0.        ],
       [0.        , 0.        , 0.        ]])

In [35]:
features = r_mat_inv.dot(matrix)
features.toarray()

array([[1.        , 0.        , 0.        ],
       [0.        , 0.33333333, 0.66666667],
       [0.        , 0.        , 0.        ]])

csr_matrix:data是矩阵的非零值，indices是和非零值一一对应的所在行的列位置，indptr是总计非零值的个数，第一个元素默认是0，从第二个元素开始记录每行非零的值个数，这个再结合data按照顺序就可以确定一个稀疏矩阵

In [43]:
indptr = np.array([0, 2, 4, 6])
indices = np.array([0, 2, 1, 2, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
matrix = sp.csr_matrix((data, indices, indptr), shape=(3, 3))
matrix.todense()

matrix([[1, 0, 2],
        [0, 3, 4],
        [0, 5, 6]])

lil_matrix:可以切片，更新数据，非常灵活。

In [45]:
data = [1, 1, 2]
row = [0, 1, 1]
col = [0, 1, 2]
matrix = sp.coo_matrix((data, (row, col)), shape=(3, 3))
matrix.todense()


matrix([[1, 0, 0],
        [0, 1, 2],
        [0, 0, 0]])

In [46]:
matrix = matrix.tolil()
matrix[0,2] = matrix[1,1]
matrix.todense()

matrix([[1, 0, 1],
        [0, 1, 2],
        [0, 0, 0]], dtype=int32)

In [48]:
matrix[[0,2]] = matrix[1]
matrix.todense()

matrix([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]], dtype=int32)